In [ ]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the paths
train_dir = '/content/drive/MyDrive/Oocyte.v6i.multiclass/train'
valid_dir = '/content/drive/MyDrive/Oocyte.v6i.multiclass/valid'
test_dir = '/content/drive/MyDrive/Oocyte.v6i.multiclass/test'
num_classes = 2

In [ ]:
# Load CSV files
train_csv = pd.read_csv(os.path.join(train_dir, '/content/drive/MyDrive/Oocyte.v6i.multiclass/train/_classes.csv'))
valid_csv = pd.read_csv(os.path.join(valid_dir, '/content/drive/MyDrive/Oocyte.v6i.multiclass/valid/_classes.csv'))
test_csv = pd.read_csv(os.path.join(test_dir, '/content/drive/MyDrive/Oocyte.v6i.multiclass/test/_classes.csv'))

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
print("Train CSV columns:", train_csv.columns)

Train CSV columns: Index(['filename', ' bad', ' good'], dtype='object')


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=train_dir,
    x_col="filename",
    y_col=[" bad", " good"],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw')

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_csv,
    directory=valid_dir,
    x_col="filename",
    y_col=[" bad", " good"],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_csv,
    directory=test_dir,
    x_col="filename",
    y_col=[" bad", " good"],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw')

Found 957 validated image filenames.
Found 78 validated image filenames.
Found 34 validated image filenames.


In [ ]:
# Build ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=valid_generator,
    validation_steps=len(valid_generator))

Epoch 1/10
30/30 [==============================] - 60s 702ms/step - loss: 0.4580 - accuracy: 0.7722 - val_loss: 0.7351 - val_accuracy: 0.4487
Epoch 2/10
30/30 [==============================] - 19s 634ms/step - loss: 0.4075 - accuracy: 0.8046 - val_loss: 0.8346 - val_accuracy: 0.4487
Epoch 3/10
30/30 [==============================] - 20s 643ms/step - loss: 0.4014 - accuracy: 0.8098 - val_loss: 0.9743 - val_accuracy: 0.4487
Epoch 4/10
30/30 [==============================] - 20s 640ms/step - loss: 0.3866 - accuracy: 0.8234 - val_loss: 1.3480 - val_accuracy: 0.4487
Epoch 5/10
30/30 [==============================] - 20s 646ms/step - loss: 0.3707 - accuracy: 0.8297 - val_loss: 0.8514 - val_accuracy: 0.4487
Epoch 6/10
30/30 [==============================] - 22s 720ms/step - loss: 0.3618 - accuracy: 0.8412 - val_loss: 0.8284 - val_accuracy: 0.4487
Epoch 7/10
30/30 [==============================] - 19s 632ms/step - loss: 0.3436 - accuracy: 0.8464 - val_loss: 0.8469 - val_accuracy: 0.4487

In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator, steps=len(train_generator))
val_loss, val_accuracy = model.evaluate(valid_generator, steps=len(valid_generator))
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Validation accuracy: {val_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")

2/2 [==============================] - 0s 23ms/step - loss: 0.6611 - accuracy: 0.6765
Train accuracy: 0.51
Validation accuracy: 0.45
Test accuracy: 0.68
